In [1]:
import netCDF4 as nc
import numpy as np
import numpy.ma as ma
import sys
import os
import argparse
import xarray as xr 
import matplotlib
%matplotlib inline

# parser = argparse.ArgumentParser(
#         description='generate daily surface C grid velocites files from a years worth of B grid daily velocities ')

# parser.add_argument('--C_grid_spec', metavar='C_grid_spec', default='/archive/net/CM2.6/regrid/CM2p6_half_len.nc',
#                     help='the netcdf file containing the C grid half-face lengths')

# parser.add_argument('--B_vel', type=str, metavar='B_vel', default='/archive/net/02020101_surf_vel.nc',
#                     help='netcdf paths to u and v containing a years worth of B grid 5 day velocities')

# parser.add_argument('--out', type=str, metavar='out', default='/archive/net/CM2p6_regrid/',
#                     help='the output folder for C grid 5 day velocity fields')

# parser.add_argument('--depth', type=int, metavar='depth', default=0,
#                     help='the depth at which to regrid')

args = parser.parse_args()

In [2]:
def roll_values(ds, roll_dict):
    true_coords = ds.coords 
    ds_rolled = ds.roll(**roll_dict)
    for coord in true_coords:
        ds_rolled.coords[coord] = ds.coords[coord]
    return ds_rolled

In [3]:
# B_vel = '/archive/net/02020101_surf_vel.nc'
# C_grid_spec = '/archive/net/CM2.6/regrid/CM2p6_half_len.nc'
# out = '/home/net/CM2p6_regrid/'
# depth = 0

B_vel = args.B_vel
C_grid_spec = args.C_grid_spec
out = args.out
# depth = 0

In [4]:
half_len_data = xr.open_dataset(C_grid_spec)
lat_T = half_len_data.grid_y_T[0:2107]
lon_T = half_len_data.grid_x_T[...]
du_Eface_N = half_len_data.ds_21_22_T[0:2107, :]
du_Eface_S = half_len_data.ds_20_21_T[0:2107, :]

In [5]:
vel_B = xr.open_dataset(B_vel, decode_times=False)
lat_U = vel_B.yu_ocean[0:2107]
lon_U = vel_B.xu_ocean[...]

In [6]:
#construct the C grid U grid  
xu_c = xr.zeros_like(lon_U)
xu_c.values = np.roll(lon_U.values, 1) #xu_c[i] = lon_U[i-1] 
# xu_c[0] = -280.0

xu_c.coords['xu_ocean'].values = xu_c.values
xu_c.attrs['long_name'] = 'ucell longitude on C grid'
xu_c = xu_c.rename({'xu_ocean': 'xu_c'})

yu_c = lat_T
yu_c.coords['grid_y_T'].values = yu_c.values
yu_c = yu_c.rename({'grid_y_T': 'yu_c'})
yu_c.attrs['long_name'] = 'ucell latitude on C grid'

#construct the C grid V grid  
xv_c = lon_T
xv_c.coords['grid_x_T'].values = xv_c.values
xv_c = xv_c.rename({'grid_x_T': 'xv_c'})
xv_c.attrs['long_name'] = 'vcell longitude on C grid'


yv_c = xr.zeros_like(lat_U)
yv_c.values = np.roll(lat_U.values, 1) #xu_c[i] = lon_U[i-1] 

yv_c.coords['yu_ocean'].values = yv_c.values
yv_c = yv_c.rename({'yu_ocean': 'yv_c'})
yv_c.attrs['long_name'] = 'vcell latitude on C grid'

#adds in a row of C grid V points beneath the existing Tcell row 
yv_c[0] = lat_U[0] + lat_U[0] - lat_U[1] 

In [7]:
%%time
slab_indices = {'time': slice(0,21), 'yu_ocean': slice(0, 2107)}
u_B_slab = vel_B.usurf.isel(**slab_indices)
u_B_slab = u_B_slab.fillna(0.0)

u_C_slab = xr.zeros_like(u_B_slab)
u_C_slab = u_C_slab.rename({'xu_ocean': 'xu_c'})
u_C_slab.coords['xu_c'] = xu_c
u_C_slab = u_C_slab.rename({'yu_ocean': 'yu_c'})
u_C_slab.coords['yu_c'] = yu_c
u_C_slab.attrs['long_name'] = 'i-surface current on SW convention C grid'


# 	for i in range(LON_POINTS):
# 	    for j in range(1,LAT_POINTS):
# 	        u_C[i,j] = ( (du_Eface_N[i-1, j]*u_B[i-1, j] + du_Eface_S[i-1, j]*u_B[i-1, j-1]) \
# 	                    /(du_Eface_N[i, j-1] + du_Eface_S[i, j-1] ) )  

roll_du_Eface_N = {'grid_x_T': 1, 'grid_y_T': 0}
roll_du_Eface_S = {'grid_x_T': 1, 'grid_y_T': 0}
roll_u_B_N = {'xu_ocean': 1, 'yu_ocean': 0}
roll_u_B_S = {'xu_ocean': 1, 'yu_ocean': 1}

du_Eface_N_roll = roll_values(du_Eface_N, roll_du_Eface_N).values
du_Eface_S_roll = roll_values(du_Eface_S, roll_du_Eface_S).values 
numerat_1 = np.multiply(du_Eface_N_roll, roll_values(u_B_slab, roll_u_B_N).values) 
numerat_2 = np.multiply(du_Eface_S_roll, roll_values(u_B_slab, roll_u_B_S).values) 
denom = du_Eface_N_roll + du_Eface_S_roll

u_C_slab.values = np.divide(numerat_1 + numerat_2, denom)

CPU times: user 4.15 s, sys: 21.2 s, total: 25.3 s
Wall time: 25.3 s


In [8]:
%%time
slab_indices = {'time': slice(0,21), 'yu_ocean': slice(0, 2107)}
v_B_slab = vel_B.vsurf.isel(**slab_indices)
v_B_slab = v_B_slab.fillna(0.0)


v_C_slab = xr.zeros_like(v_B_slab)
v_C_slab = v_C_slab.rename({'xu_ocean': 'xv_c'})
v_C_slab.coords['xv_c'] = xv_c
v_C_slab = v_C_slab.rename({'yu_ocean': 'yv_c'})
v_C_slab.coords['yv_c'] = yv_c
v_C_slab.attrs['long_name'] = 'j-surface current on SW convention C grid'


# 	for i in range(LON_POINTS):
# 	    for j in range(LAT_POINTS):
# 	        v_C[i,j] = (v_B[i-1, j-1] + v_B[i, j-1])/2
            
roll_1 = {'xu_ocean': 1, 'yu_ocean': 1}
roll_2 = {'xu_ocean': 0, 'yu_ocean': 1}

v_C_slab.values = (roll_values(v_B_slab, roll_1).values + roll_values(v_B_slab, roll_2).values)/2

CPU times: user 3.03 s, sys: 13.5 s, total: 16.5 s
Wall time: 16.6 s


In [16]:
vel_C = xr.merge([u_C_slab, v_C_slab])

In [17]:
vel_C

<xarray.Dataset>
Dimensions:  (time: 21, xu_c: 3600, xv_c: 3600, yu_c: 2107, yv_c: 2107)
Coordinates:
  * time     (time) float64 7.342e+04 7.342e+04 7.342e+04 7.342e+04 ...
  * xu_c     (xu_c) float64 80.0 -279.9 -279.8 -279.7 -279.6 -279.5 -279.4 ...
  * yu_c     (yu_c) float64 -81.11 -81.07 -81.02 -80.98 -80.94 -80.9 -80.86 ...
  * xv_c     (xv_c) float64 -280.0 -279.9 -279.8 -279.6 -279.5 -279.5 -279.4 ...
  * yv_c     (yv_c) float64 -81.13 -81.09 -81.05 -81.0 -80.96 -80.92 -80.88 ...
Data variables:
    usurf    (time, yu_c, xu_c) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    vsurf    (time, yv_c, xv_c) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...

In [12]:
v_C_slab.to_netcdf('/home/net/test_v.nc')

In [14]:
# verify_u = '/archive/net/CM2.6/regrid/regrid_2020_90_day/U0000000020.nc'
# ds_u_verif = xr.open_dataset(verify_u, decode_times=False)
# verify_patch_u = {'time': 0, 'xu_c': 200, 'yu_c': 1200}
# print ds_u_verif.isel(**verify_patch_u).u
# print u_C_slab.sel(time=ds_u_verif.time).isel(**verify_patch_u)

<xarray.DataArray 'u' ()>
array(0.0592631995677948, dtype=float32)
Coordinates:
    time     float32 73435.5
    yu_c     float32 -4.44554
    xu_c     float32 -260.0
Attributes:
    units: m/s
    long_name: zonal velocity on C grid
<xarray.DataArray 'usurf' ()>
array(0.05925509188479734)
Coordinates:
    time     float64 7.344e+04
    xu_c     float64 -260.0
    yu_c     float64 -4.446
Attributes:
    long_name: i-surface current on SW convention C grid
    units: m/sec
    valid_range: [-10.  10.]
    cell_methods: time: mean
    time_avg_info: average_T1,average_T2,average_DT
    coordinates: geolon_c geolat_c


In [15]:
# verify_v = '/archive/net/CM2.6/regrid/regrid_2020_90_day/V0000000020.nc'
# ds_v_verif = xr.open_dataset(verify_v, decode_times=False)
# verify_patch_v = {'time': 0, 'xv_c': 1200, 'yv_c': 1200}
# print ds_v_verif.isel(**verify_patch_v).v
# print v_C_slab.sel(time=ds_u_verif.time).isel(**verify_patch_v)

<xarray.DataArray 'v' ()>
array(0.18643414974212646, dtype=float32)
Coordinates:
    time     float32 73435.5
    yv_c     float32 -4.49538
    xv_c     float32 -159.95
Attributes:
    units: m/s
    long_name: meridional velocity on C grid
<xarray.DataArray 'vsurf' ()>
array(0.18643414229154587)
Coordinates:
    time     float64 7.344e+04
    xv_c     float64 -159.9
    yv_c     float64 -4.495
Attributes:
    long_name: j-surface current on SW convention C grid
    units: m/sec
    valid_range: [-10.  10.]
    cell_methods: time: mean
    time_avg_info: average_T1,average_T2,average_DT
    coordinates: geolon_c geolat_c
